<a href="https://colab.research.google.com/github/PyAshishMhatre/DigitalMarketing-Algorithms-Project/blob/main/QA_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing Libraries
!pip install pytube -q
!pip install git+https://github.com/openai/whisper.git -

In [4]:
from pytube import YouTube
import whisper
from tqdm import tqdm

In [62]:
model = whisper.load_model('tiny')

# Function to download video and get transcriptions
def get_transcriptions(url, model):
    yt_video = YouTube(url)
    stream = yt_video.streams.filter(only_audio=True)
    stream = stream.first()
    stream.download(filename="test.mp4")
    output = model.transcribe("test.mp4")

    filename = "test.txt" # the name of the file to be saved

    with open(filename, "w") as file:
      file.write(output["text"])
    return 

In [66]:
# List of 5 video URLs
video_urls = [#"https://www.youtube.com/watch?v=blbvVUxD41Q&ab_channel=Locust%26WildHoney",
              #"https://www.youtube.com/watch?v=9hktZEc3Vhs&ab_channel=STYLEDBYNATE",
              #"https://www.youtube.com/watch?v=C9nVeYwS_8E&ab_channel=Men%27sFashionFiles",
              #"https://www.youtube.com/watch?v=7fJcrPjAa1I&ab_channel=UniqloReviews",
              "https://www.youtube.com/watch?v=YtjHtPySBAA&ab_channel=HarryHas"]

# Loop through each video URL and get transcriptions
for url in tqdm(video_urls):
  get_transcriptions(url, model)

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
100%|██████████| 1/1 [01:19<00:00, 79.16s/it]


In [ ]:
!pip install langchain
!pip install openai
!pip install chromadb

In [94]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from typing import List
from langchain.chains import RetrievalQA
from langchain.schema import Document
import os

In [90]:
os.environ['OPENAI_API_KEY'] = ""

In [71]:
loader = DirectoryLoader('./Document', glob="**/*.txt", loader_cls=TextLoader, show_progress=True)

In [72]:
docs = loader.load()

100%|██████████| 9/9 [00:00<00:00, 5036.52it/s]


In [73]:
len(docs)

9

In [78]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

In [91]:
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(texts, embeddings)

In [96]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=vectordb.as_retriever(), veborse)

In [106]:
query = "How is shirt fitting describe for a particular shirt explain to 5 year old ?"


In [107]:
result = qa.run(query)

In [108]:
result

' The particular shirt is described as having a boxy fit, feeling thinner, more rough, and more restrictive than it used to be. It has a tighter hem at the bottom, and the sleeve fits the same as it used to. To explain this to a 5 year old, you could say that the shirt fits more snugly than it used to, and it may feel a bit scratchy.'